This script can currently run analysis and create plots for VNA, EYEBERT, and DC resistance. Data should be organized with channels running in the columns and different cable numbers running in the rows. The user should have the specific rows listed out that correspond to the specific lengths as the program will ask the user for these values so it can slice the data correctly.
Second block is for DC resistance and third block is for EYE and VNA because there is 10 columns for DC (one for each channel) but there are 5 columns for EYE and VNA (2 for each channel, positive and negative)
Blocks will return arrays with average resistance values over lengths or channels depending on what the user wants along with error values for each value

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pprint import pprint
import random

# Call in your data, either VNA, DC4pt, SCC, or EYEBERT data
# Data should be arranged in the format described below
        # Columns should separate channels, with first two columns representing CMD
        # and last two representing D3 (or D2 in TFPX case)
        # Rows should separate cable numbers, with different slices representing
        # different lengths
excel = pd.read_excel (r'C:\Users\bruen\Coding_Projects\Data_Analysis\DC4ptResistance\DC4pt_Data.xlsx')

data1 = excel.to_numpy() #Convert to numpy array

# Which test produced this data set?
data_type = input("Which test produced this data? Type VNA,SCC,DC,or EYE: ")
    

This next block is for the DC 4 port resistance data.
Returns arrays for average resistance over certain lengths (avg_lengths) and
average resistance over certain channels (avg_channels)
Along with error arrays for each group of measurements (error_lengths,error_channels)
Also defines channel and length arrays for future plotting
**Works for TFPX as well but that data set only contains one length 

In [ ]:
if data_type == 'DC':
    # Function that slices data by rows according to two parameters, start and end row
    # Then calculates the mean of that row
    def averaging(a,b):
        sli = data1[a:b+1,0:11]
        average = np.mean(sli)
        return average
    # Function that calculates the error in the slices
    def error_calc(data):
        error = np.std(data) / np.sqrt((data.size))
        return error
    # Later I want to add a scatter plot, so one function should not calculate mean
    def slicing(a,b):
        sli = data1[a:b+1,0:11]
        return sli

    i = 0 # Define index for stepping through our channel slices (will be incremented by 2)
    rows_or_cols = input("Would you like to split the data into its lengths or channels? (1 for lengths or 2 for channels):")
    if rows_or_cols == '2': # Splitting into channels, but we still need to split into lengths first
        type_link = int(input("Is this a 15 pin or 33 pin connection? (Type 1 for 33, 2 for 15):"))
        # The next 3 questions are asked because resistance varies with length so in order to plot vs its channels 
        # we need the data all from the same length
        name = str(input("What is the length and tyype of this group of data? (Ex: Type1: One Point Six"))
        beg = int(input("What row does your length start in? (Type integer):"))
        end = int(input("What row does your length end in? (Type integer):"))
        if type_link == 1: # 33 pin connection
            channels = [1,2,3,4,5]
            avg_channels = [0,1,2,3,4] # Initialize our lists to store values in
            error_channels = [0,1,2,3,4]
            l = 0 # Define our index for placing our values in the lists
            while i < 5: # It is multiplied by 2 because we are incrementing our "i" index by 2
                avg_channels[l] = np.mean(slicing(beg,end)[:,i:i+2]) # Appending our average value to the list
                error_channels[l] = error_calc(slicing(beg,end)[:,i:i+2]) # Appending our error value to the list
                i += 2
                l += 1
        # Same code as the above if statement just just the lists are shorter since 15 pin connection only has
        # 4 channels instead of 5
        if type_link == 2: # Testing a 15 pin connection
            channel = [1,2,3,4]
            avg_channels = [0,1,2,3]
            error_channels = [0,1,2,3]
            l = 0
            while i < 4:
                channel = [1,2,3,4]
                avg_channels = [0,1,2,3]
                error_channels = [0,1,2,3]
                avg_channels[l] = np.mean(slicing(beg,end)[:,i:i+2])
                error_channels[l] = error_calc(slicing(beg,end)[:,i:i+2])
                i += 2
                l += 1
        j = 0
        k = 0
    if rows_or_cols == '1':
        no_of_slices = int(input("How many different lengths does your data have? (Type an integer):"))
        avg_lengths = random.sample(range(10, 30), no_of_slices)
        error_lengths = random.sample(range(10,30), no_of_slices)
        while i < no_of_slices:
            beg = int(input("What row does your length start in? (Type integer):"))
            end = int(input("What row does your length end in? (Type integer):"))
            avg_lengths[i] = averaging(beg,end)
            error_lengths[i] = error_calc(slicing(beg,end))
            i += 1


This next block is for VNA and EYE BERT tests. It returns similair arrays as the one above. DO NOT USE THIS BLOCK TO PLOT EYE BERT VS CHANNELS (because of its linear dependence)
VNA is linearly independent so we can plot every single measurement vs its channels and there is no discrepancies

In [ ]:
if data_type == 'VNA' or 'EYE':
        # Function that slices data by rows according to two parameters, start and end row
# Then calculates the mean of that row
    def averaging(a,b):
        sli = data1[a:b+1,0:5]
        average = np.mean(sli)
        return average
    # Function that calculates the error in the slices
    def error_calc(a,b):
        sli = data1[a:b+1,0:5]
        error = np.std(sli) / np.sqrt((sli.size))
        return error
    # Later I want to add a scatter plot, so one function should not calculate mean
    def slicing(a,b):
        sli = data1[a:b+1,0:5]
        return sli
    
    i = 0
    rows_or_cols = input("Would you like to split the data into its lengths or into its channels? (1 for lengths or 2 for channels):")
    if rows_or_cols == '2': # User decides to split into channels
        type_link = int(input("Is this a 15 pin or 33 pin connection? (Type 1 for 33, 2 for 15):"))
        if type_link == 1:
            CMD = np.mean(data1[:,0:1])
            D0 = np.mean(data1[:,1:2])
            D1 = np.mean(data1[:,2:3])
            D2 = np.mean(data1[:,3:4])
            D3 = np.mean(data1[:,4:5])
            avg_channels = [CMD,D0,D1,D2,D3]
            error_channels = random.sample(range(10,30),5)
            # We need an x-axis for our plot
            channels = [1,2,3,4,5]
        if type_link == 2:
            CMD = np.mean(data1[:,0:1])
            D0 = np.mean(data1[:,1:2])
            D1 = np.mean(data1[:,2:3])
            D2 = np.mean(data1[:,3:4])
            avg_channels = [CMD,D0,D1,D2]
            error_channels = random.sample(range(10,30),4)
            # We need an x-axis for our plot
            channels = [1,2,3,4]
        j = 0
        k = 0
        while j < len(avg_channels):
            error_channels[k] = error_calc(j,j+1)
            k += 1
            j += 2      
    if rows_or_cols == '1': # User decides to split into lengths
        no_of_slices = int(input("How many different lengths does your data have? (Type an integer):"))
        avg_lengths = random.sample(range(10, 30), no_of_slices)
        error_lengths = random.sample(range(10,30), no_of_slices)
        while i < no_of_slices:
            beg = int(input("What row does your length start in? (Type integer):"))
            end = int(input("What row does your length end in? (Type integer):"))
            avg_lengths[i] = averaging(beg,end)
            error_lengths[i] = error_calc(beg,end)
            i += 1
        lengths = np.array([.35,.80,1.00,1.40,1.60,1.80], dtype=float)
    